#  Select a location for café shop opening in Taichung City, Taiwan

## 1.Introduction 

### Taichung is the top 3 big cities in Taiwan, which is competeitive in food and drinks industry. I plan to analyze c urrent café shop distribution in Taichung around my home. And cluster the shops to different clusters, try to select a location for new coffee shop opening.

In [45]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation


!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize


! pip install folium==0.5.0
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


In [46]:
CLIENT_ID = 'CZTYNXU2GXW1EQJ3F0TVJILHCIHFUFALLZX22JUOLJHINLSE' # your Foursquare ID
CLIENT_SECRET = 'XXCDRT0AASH4HV1XGPJ2PPOERZSA1SPWGZJPJ030F1T4QY5V' # your Foursquare Secret
ACCESS_TOKEN = 'TSRSRFVBG3LQQC2S5PUH0L30AQRMTEMYIVMKCLDNDHJLGDSB' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 1000
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: CZTYNXU2GXW1EQJ3F0TVJILHCIHFUFALLZX22JUOLJHINLSE
CLIENT_SECRET:XXCDRT0AASH4HV1XGPJ2PPOERZSA1SPWGZJPJ030F1T4QY5V


### lagtitude and lontitude of my home

In [47]:
#Location near my home
latitude=24.150915
longitude=120.647743

## 2.Data collection

### Searching cafe shop with radius 2000m around my home

In [48]:
search_query = 'cafe'
radius = 2000
print(search_query + ' .... OK!')

cafe .... OK!


In [49]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60482453705bdd22602cd792'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Taichung',
  'headerFullLocation': 'Taichung',
  'headerLocationGranularity': 'city',
  'query': 'cafe',
  'totalResults': 61,
  'suggestedBounds': {'ne': {'lat': 24.168915018000018,
    'lng': 120.66743284295063},
   'sw': {'lat': 24.132914981999985, 'lng': 120.62805315704938}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bad7f67f964a520a6563be3',
       'name': 'Cafe Terry',
       'location': {'address': '418 Daying St.',
        'crossStreet': 'DaDunShi',
        'lat': 24.151553974069

### Get the data we need,  the list of cafe shops around my home with radius 2km.

In [50]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [51]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

<ipython-input-51-561c05f0fdd1>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,Cafe Terry,Café,24.151554,120.648708
1,THE FACTORY / mojocoffee,Café,24.155055,120.657085
2,mezamashikohi Trio (目覺三店),Café,24.155156,120.656170
3,Caffaina Coffee Gallery (卡啡那（惠來店）),Café,24.155698,120.641258
4,edia cafe,Café,24.156049,120.652248


## 3.Methodology

### Visualize with the map

In [52]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around my home

# add a red circle marker to represent my home
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='my home',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)


# add the Cafe shop as blue circle markers
for lat, lng, label in zip(nearby_venues.lat, nearby_venues.lng, nearby_venues.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

    
    
# display map
venues_map

### From above map, blue points are coffee shops. We can see the coffe shops are located on east side more than west side. Let's try to see what's the distribution of restaurant. 

### To see if the population makes the distribution like this.

### Get the data we need, the list of restaurant around my home with radius 2km.

In [53]:
search_query = 'restaurant'
radius = 2000

url2 = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)

results2 = requests.get(url2).json()

venues2 = results2['response']['groups'][0]['items']
    
nearby_venues2 = json_normalize(venues2) # flatten JSON

# filter columns
filtered_columns2 = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues2 =nearby_venues2.loc[:, filtered_columns]

# filter the category for each row
nearby_venues2['venue.categories'] = nearby_venues2.apply(get_category_type, axis=1)

# clean columns
nearby_venues2.columns = [col.split(".")[-1] for col in nearby_venues2.columns]

nearby_venues2.head()

<ipython-input-53-87d9105c0b97>:10: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues2 = json_normalize(venues2) # flatten JSON


,name,categories,lat,lng
0,Cafe Terry,Café,24.151554,120.648708
1,Bistro88,Italian Restaurant,24.150975,120.649900
2,金悅軒,Seafood Restaurant,24.150820,120.645681
3,無老鍋,Hotpot Restaurant,24.150914,120.650600
4,澄石 CHENSVEG cafe kitchen,Vegetarian / Vegan Restaurant,24.152334,120.653112


### Visualize with the map

In [54]:
# add the restaurant as green circle markers
for lat, lng, label in zip(nearby_venues2.lat, nearby_venues2.lng, nearby_venues2.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='green',
        popup=label,
        fill = True,
        fill_color='green',
        fill_opacity=0.1
    ).add_to(venues_map)
    
venues_map

### From above map, we can see that seems restaurants are distributed more comprehensive than coffee shop.
### Suppose population is not resulting in huge difference in restaurant opening

## 4.Analysis

### Since there is no detailed information of ''Likes","Tips" in Foursquare for these coffee shops in Taichung. Therefore I try to cluster the shops and find a not so crown area. 

### Using clustering method

In [55]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 4

coffee_grouped_clustering = nearby_venues.drop('name', 1)

coffee_grouped_clustering = coffee_grouped_clustering.drop('categories', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(coffee_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 3, 3, 1, 3, 2, 0, 1, 0, 3])

In [56]:
# add clustering labels
nearby_venues.insert(0, 'Cluster Labels', kmeans.labels_)

#manhattan_merged = manhattan_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
#manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

nearby_venues.head() # check the last columns!

,Cluster Labels,name,categories,lat,lng
0,2,Cafe Terry,Café,24.151554,120.648708
1,3,THE FACTORY / mojocoffee,Café,24.155055,120.657085
2,3,mezamashikohi Trio (目覺三店),Café,24.155156,120.656170
3,1,Caffaina Coffee Gallery (卡啡那（惠來店）),Café,24.155698,120.641258
4,3,edia cafe,Café,24.156049,120.652248


## 5.Results&Conclusion

In [57]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon,cluster in zip(nearby_venues['lat'], nearby_venues['lng'], nearby_venues['Cluster Labels']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### From map above, we can see the light blue and purple area are not so densed of coffee shops distribution.

### Actually a better way for analysis would be taking data from google API and getting population data to better analyze the current situation then decide a location for coffee shop opening. The information from Foursquare is not enough for the analysis.

### According to the maps, I will choose the purple area for new coffee shop opening, since it is the area with City Hall, also a new developed area with shopping mall and residential district . Should be easier have parling lot and expect to have more people  coming.